# Mangrove tiles

We need create vector tiles for the Mangrove restoration and typology data provided by the client [here](https://github.com/Vizzuality/mangrove-atlas/issues/331).  
The original link to the source data is [here](https://tnc.app.box.com/s/8abiem7o8lyd9x1pvpe2yn2mdibgpcqk).  
The source data is an arcgis GDB and the data is stored in gcs bucket.  
The steps this pipe will take are:  
Pre.- script to move data from their original location to the gcs bucket.  
1.- Get data.  
2.- Tranform original data to geojson with (mapshaper, geopandas, ...)  
3.- Generate the mbtile file with tippecanoe  
4.- Upload the mbtile file to gcs and mapbox as a tileset.  
5.- Comunicate to Design and FE team the tileset is ready.  

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import fiona


## 1. Download raw data
Load data from Google Cloud Storage

In [10]:

data_url = 'https://storage.googleapis.com/mangrove_atlas/widget_data'
vector_file = 'Restoration_Update_20221201.gdb'
file_path =f'{data_url}/Typology_and_Restoration_Potential/Data/{vector_file}'

# Loading from GCS failed
file_path = '../../../../data/Typology_and_Restoration_Potential/Data/Restoration_Update_20221201.gdb'
layer = gpd.read_file(file_path, driver='FileGDB', layer=0)
layer.head()

,OBJECTID,Class,ID,Type,Country,Region,Max_Area_20_ha,Area_loss_ha,Area_loss_pct,Rest_Area_Loss,...,AGB,Fish_Score,Fish_Score_Inv,Loss_Driver,Min_Score,Max_Score,Crab,Bivalve,Shrimp,geometry
0,1,Delta,30028,Delta_30028,Madagascar,None,18276.018627,1439.322931,8.0,1215.253234,...,44405.018810,9.199026e+07,1.735094e+08,Extreme Weather,80.578292,83.454375,2.446594e+07,0.0,1.490435e+08,"MULTIPOLYGON (((44.49711 -19.93222, 44.49689 -..."
1,2,Delta,50000,Delta_50000,Malaysia,None,26236.287075,793.173736,3.0,523.385875,...,105072.355871,6.792331e+07,1.120986e+08,Commodities,69.349536,87.600000,3.378470e+05,0.0,1.117608e+08,"MULTIPOLYGON (((118.91244 5.34022, 118.91267 5..."
2,3,Delta,50001,Delta_50001,Indonesia,None,105197.959853,2712.808923,3.0,2460.690041,...,683469.905996,1.949940e+08,3.298283e+08,Commodities,71.845840,89.927338,1.099365e+06,0.0,3.287290e+08,"MULTIPOLYGON (((117.75422 3.76556, 117.75422 3..."
3,4,Delta,70000,Delta_70000,Brazil,None,186915.778395,10409.261834,6.0,2514.855617,...,140900.620655,4.526629e+07,8.450866e+07,Erosion,85.533365,89.239620,8.450866e+07,0.0,0.000000e+00,"MULTIPOLYGON (((-44.80889 -3.37644, -44.80889 ..."
4,5,Delta,70001,Delta_70001,Brazil,None,230156.812403,10676.953931,5.0,3198.093967,...,138312.748494,7.065099e+07,1.199932e+08,Erosion,86.223525,89.229099,1.199932e+08,0.0,0.000000e+00,"MULTIPOLYGON (((-44.65489 -2.48711, -44.65489 ..."


# 2. Transform to geoJSON

Change CRS to WSG84

In [11]:
layer.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [5]:
layer = layer.to_crs('EPSG:4326')
layer.crs
#2022-12 data update is already WSG84

In [12]:
out_path = '../../../../data/Typology_and_Restoration_Potential/Data/MOW_Global_Mangrove_Restoration_202212.geojson'

layer.to_file(out_path, driver="GeoJSON") 

## 3. Generate the mbtile file with tippecanoe  


In [14]:
in_file = '../../../../data/Typology_and_Restoration_Potential/Data/MOW_Global_Mangrove_Restoration_202212.geojson'
out_file = '../../../../data/Typology_and_Restoration_Potential/Data/MOW_Global_Mangrove_Restoration_202212.mbtiles'
!tippecanoe -zg -o $out_file --drop-densest-as-needed --force $in_file
print('ALL DONE!')

For layer 0, using name "MOW_Global_Mangrove_Restoration_202212"
3985 features, 234390395 bytes of geometry, 655920 bytes of separate metadata, 885597 bytes of string pool
Choosing a maxzoom of -z1 for features about 175111 feet (53374 meters) apart
Choosing a maxzoom of -z11 for resolution of about 137 feet (42 meters) within features
  99.9%  11/1560/932   
ALL DONE!


ALL DONE!


### Optional: visualize tiles with mbview 

In [15]:
#!npm install -g @mapbox/mbview
out_file = '../../../data/Typology_and_Restoration_Potential/Data/MOW_Global_Mangrove_Restoration.mbtiles'
!mbview $out_file

missing access token, try `export MAPBOX_ACCESS_TOKEN=...`


## 4. Upload the mbtile file to gcs and mapbox as a tileset.  

## 5. Comunicate to Design and FE team the tileset is ready.  